In [1]:
from dataset import dataset
from attention import Attention2D
import utils as utils
import datetime
import torch
import torch.nn as nn
from os import mkdir, getcwd
from os.path import join, dirname

In [3]:
rot, t = utils.camTrans()
model = Attention2D() #.to(device)
model_path = '../checkpoints/spatial-softmax/20210715-175329/checkpoint-2000.pth'
md = torch.load(model_path)
ts = md['model']

In [7]:
model.load_state_dict(md['model'])

RuntimeError: Error(s) in loading state_dict for Attention2D:
	Missing key(s) in state_dict: "pose_regress.model.2.weight", "pose_regress.model.2.bias", "pose_regress.model.3.running_mean", "pose_regress.model.3.running_var", "pose_regress.model.5.weight", "pose_regress.model.5.bias". 
	Unexpected key(s) in state_dict: "pose_regress.model.4.weight", "pose_regress.model.4.bias", "pose_regress.model.4.running_mean", "pose_regress.model.4.running_var", "pose_regress.model.4.num_batches_tracked". 
	size mismatch for feature.model.0.weight: copying a param with shape torch.Size([32, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 4, 3, 3]).
	size mismatch for pose_regress.model.3.weight: copying a param with shape torch.Size([21, 96]) from checkpoint, the shape in current model is torch.Size([48]).
	size mismatch for pose_regress.model.3.bias: copying a param with shape torch.Size([21]) from checkpoint, the shape in current model is torch.Size([48]).